## Astrogrism example notebook

This notebook shows an example of using the `GrismObs` class, the core
object of the `astrogrism` package, to transform between the available
coordinate frames.

In [ ]:
# First, we import the necessary modules

import pathlib

import astropy.units as u

from astrogrism import GrismObs

The following cell assumes that you are running this notebook
in place in its location in your local copy of the `astrogrism`
repository. If you're running it in a different location, you
will need to provide the path to the test file referenced.

In [ ]:
pkg_dir = pathlib.Path('.').absolute().parent
acs_file = str(pkg_dir / 'astrogrism/tests/data/acs_test_file.fits')

To read in a grism observation, simply provide the path to the file as the
first argument to the `GrismObs` class:

In [ ]:
grism_obs = GrismObs(acs_file)

The main purpose of the `GrismObs` class is to provide access to the tranforms
between the geometric frames, in order to determine e.g. where a pixel on the
direct image would be dispersed to on the dispersed grism image for a 
given wavelength. You can see the available frames as follows:

In [ ]:
grism_obs.geometric_transforms["CCD1"].available_frames

Note that the file we're using is an ACS observation, which has two CCDs and thus
requires specifying which CCD we're interested in. WFC3 UVIS also has two CCDs and 
thus requires the same specification of "CCD1" or "CCD2", while WFC3 IR only
has one CCD. To get the available frames for WFC3 IR you would call 
`grism_obs.geometric_transforms.available_frames` (note the lack of `["CCD1"]`), and
equivalently leave out the CCD specification in the calls below.

To actually get the transform we want, we use the following command:

In [ ]:
direct_to_grism = grism_obs.geometric_transforms["CCD1"].get_transform("detector", "grism_detector")

The inputs to this function can be seen by calling the following. Note that x, y, and order are 
unitless, while `wavelength` can be specified as an astropy `Quantity` object in order to 
ensure that the units are treated properly.

In [ ]:
direct_to_grism.inputs

Here we see where pixel (800, 1000) on the direct detector would be dispersed
for the first order trace at 0.7 microns.

In [ ]:
direct_to_grism(1024, 2048.0, 0.7*u.um, 1)

The reverse transform takes the coordinates on both the dispersed and direct image, along
with the order, and returns the wavelength that would be dispersed from point on the direct
image to the specified point on the grism image.

In [ ]:
grism_to_direct = grism_obs.geometric_transforms["CCD1"].get_transform("grism_detector", "detector")
grism_to_direct.inputs

In [ ]:
# Note that we simply copy-paste the output of direct_to_grism for this input
grism_to_direct(1084.130391789169, 2044.4123782198496, 1024.0, 2048.0, 1.0)

One thing to keep in mind here is that the wavelength does not round trip exactly
between the dispersed/non-dispersed images, and that the accuracy varies depending
on how good the calibration fits used are at a given location on the chip. The HST
instrument documentation has information of which regions of the chips are recommended 
for use. 

For completeness, we also show transforming between world (sky) coordinates and the
direct and dispersed image frames. Note that currently, the world coordinate inputs
must be specified as right ascension and declination in decimal degrees. We specify the 
wavelength in Angstrom below instead of micron to demonstrate that, because the wavelength 
is an astropy `Quantity` with units attached, any necessary conversions are done internally
in the `astrogrism` code.

In [ ]:
world_to_direct = grism_obs.geometric_transforms["CCD1"].get_transform("world", "detector")
world_to_grism = grism_obs.geometric_transforms["CCD1"].get_transform("world", "grism_detector")

In [ ]:
world_to_direct(264.0677510637033, -32.91199329438908, 7000*u.AA, 1.0)

In [ ]:
world_to_grism(264.0677510637033, -32.91199329438908, 7000*u.AA, 1.0)

Note that the output for the grism image frame includes both the direct image and grism image coordinates, in the format (grism_x, grims_y, direct_x, direct_y, order). This is because the output for any geometric transform is the same as the input for the reverse transform. 

### Accessing the original data

While the geometric transforms are the main point of the `GrismObs` class,
you can also access the original data (as read by the `astropy.io.fits` machinery)
through the object via the `grism_image` and `grism_header` attributes:

In [ ]:
grism_obs.grism_image

In [ ]:
grism_obs.grism_header